<a href="https://colab.research.google.com/github/shenzhun/machine-learning-brushup/blob/master/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gensim: Word2Vector

In [0]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:

  def __iter__(self):
    corpus_path = datapath('lee_background.cor')
    for line in open(corpus_path):
      yield utils.simple_preprocess(line)

In [0]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, size=10, min_count=5, workers=5)

In [41]:
model.evaluate_word_pairs(datapath('wordsim353.tsv'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `evaluate_word_pairs` (Method will be removed in 4.0.0, use self.wv.evaluate_word_pairs() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


((0.1880701204557092, 0.15014590209321094),
 SpearmanrResult(correlation=0.12007781094053302, pvalue=0.36078766671922924),
 83.0028328611898)

In [39]:
vec_king = model.wv['king']
print(vec_king)

[ 0.02192395  0.14082208  0.10967366  0.12967958  0.02253971  0.05671262
  0.23116086  0.1787115  -0.00316649  0.15387657]


In [37]:
for i, word in enumerate(model.wv.vocab):
  if i == 10:
    break
  print(word)

hundreds
of
people
have
been
forced
to
their
homes
in


# PyTorch: Embedding and Lookup

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [4]:
word_to_idx = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2, 5) # pretrained embedding model
lookup_tensor = torch.tensor([word_to_idx["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229]],
       grad_fn=<EmbeddingBackward>)


# N-Gram Language Modeling

In [0]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [10]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) - 2)]
print(trigrams[:3])

vocab = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [0]:
class NGramLanguageModeler(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size * embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)
  
  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))
    out = F.relu(self.linear1(embeds))
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dim=1)
    return log_probs
  
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [26]:
for epoch in range(10):
  total_loss = 0
  for context, target in trigrams:
    context_idxs = torch.tensor([word_to_idx[w] for w in context], dtype=torch.long)
    model.zero_grad()

    log_probs = model(context_idxs)

    loss = loss_function(log_probs, torch.tensor([word_to_idx[target]], dtype=torch.long))

    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  losses.append(total_loss)
print(losses)

[524.0827550888062, 521.2776417732239, 518.4930958747864, 515.7282702922821, 512.9818377494812, 510.2534718513489, 507.5416853427887, 504.8449203968048, 502.162556886673, 499.4929676055908, 496.8344533443451, 494.1861078739166]
